# Disease Tagging Tutorial

In this example, we'll be writing an application to extract *mentions of* diseases from Pubmed abstracts, using annotations from the [BioCreative CDR Challenge](http://www.biocreative.org/resources/corpora/biocreative-v-cdr-corpus/).  This tutorial, which has 5 parts, walks through the process of constructing a model to classify _candidate_ disease mentions as either true (i.e., that it is truly a mention of a disease) or false.

## Part IV: Training a Model with Data Programming

In this part of the tutorial, we will train a statistical model to differentiate between true and false `Disease` mentions.

We will train this model using _data programming_, and we will **ignore** the training labels provided with the training data. This is a more realistic scenario; in the wild, hand-labeled training data is rare and expensive. Data programming enables us to train a model using only a modest amount of hand-labeled data for validation and testing. For more information on data programming, see the [NIPS 2016 paper](https://arxiv.org/abs/1605.07723).

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os

# Note: We run automated tests on this tutorial to make sure that it is always up to date! 
# However, certain interactive components cannot currently be tested automatically, and will 
# be skipped with if-then statements using the variable below
AUTOMATED_TESTING = os.environ.get('TESTING') is not None

import numpy as np
from snorkel import SnorkelSession
session = SnorkelSession()

We repeat our definition of the `Disease` `Candidate` subclass from Parts II and III.

In [ ]:
from snorkel.models import candidate_subclass

ChemicalDisease = candidate_subclass('Disease', ['disease'])

## Loading `CandidateSet` objects

We reload the training and development `CandidateSet` objects from the previous parts of the tutorial.

In [ ]:
from snorkel.models import CandidateSet

train = session.query(CandidateSet).filter(CandidateSet.name == 'CDR Training Candidates').one()
dev = session.query(CandidateSet).filter(CandidateSet.name == 'CDR Development Candidates').one()

## Automatically Creating Features
Recall that our goal is to distinguish between true and false mentions of chemical-disease relations. To train a model for this task, we first embed our `ChemicalDisease` candidates in a feature space.

In [ ]:
from snorkel.annotations import FeatureManager

feature_manager = FeatureManager()

We can create a new feature set:

In [ ]:
%time F_train = feature_manager.create(session, train, 'Train Features')

**OR** if we've already created one, we can simply load as follows:

In [ ]:
%time F_train = feature_manager.load(session, train, 'Train Features')

Note that the returned matrix is a special subclass of the `scipy.sparse.csr_matrix` class, with some special features which we demonstrate below:

In [ ]:
F_train

In [ ]:
F_train.get_candidate(0)

In [ ]:
F_train.get_key(0)

## Creating Labeling Functions
Labeling functions are a core tool of data programming. They are heuristic functions that aim to classify candidates correctly. Their outputs will be automatically combined and denoised to estimate the probabilities of training labels for the training data.

In [ ]:
DATA_ROOT = os.environ['SNORKELHOME'] + '/tutorial/data/'

CTD_lower = set()
with open(DATA_ROOT + 'dicts/CTD_chemicals_diseases.tsv', 'rb') as f:
    for line in f:
        if not line.startswith("#"):
            chem, chem_id, _, disease, disease_id, evidence, _, _, _, _ = line.split('\t')
            CTD_lower.add((chem.lower(), disease.lower()))
print len(CTD_lower)

In [ ]:
from random import random
import re
from snorkel.lf_helpers import get_text_between
from snorkel.models import split_stable_id
from snorkel.lf_helpers import get_tagged_text

def LF_in_CDT(c, p_neg=0.2):
    """Match against the CDT KB, with random negative supervision as well"""
    chem    = c.chemical.get_span().lower()
    disease = c.disease.get_span().lower()
    if (chem, disease) in CTD_lower:
        return 1
    else:
        return -1 if random() < p_neg else 0

WEAK_PHRASES = ['none', 'although', 'was carried out', 'was conducted',
                'seems', 'suggests', 'risk', 'associated with', 'implicated',
               'the aim', 'to (investigate|assess|study)']

WEAK_RGX = r'|'.join(WEAK_PHRASES)

def LF_weak_assertions(c):
    return -1 if re.search(WEAK_RGX, get_tagged_text(c), flags=re.I) else 0

ANIMAL_RGX = r'mouse|mice'

def LF_animal(c):
    return -1 if re.search(ANIMAL_RGX, get_tagged_text(c), flags=re.I) else 0

def LF_cause(c):
    return 1 if re.search(r'cause', get_text_between(c), flags=re.I) else 0

def LF_in_CDT_causes(c):
    """Match against the CDT KB, with random negative supervision as well"""
    chem    = c.chemical.get_span().lower()
    disease = c.disease.get_span().lower()
    if (chem, disease) in CTD_lower:
        if re.search(r'causes', get_text_between(c), flags=re.I):
            return 1
    return 0

def LF_in_CDT_induced(c):
    chem    = c.chemical.get_span().lower()
    disease = c.disease.get_span().lower()
    if (chem, disease) in CTD_lower and re.search(r'{{A}}.{0,20}induce.{0,20}{{B}}', get_tagged_text(c), flags=re.I):
        return 1
    return 0

def LF_in_CDT_exposure(c):
    chem    = c.chemical.get_span().lower()
    disease = c.disease.get_span().lower()
    if (chem, disease) in CTD_lower and re.search(r'{{B}}.{0,20}expos.{0,20}{{A}}', get_tagged_text(c), flags=re.I):
        return 1
    return 0

def LF_in_CDT_caused_by(c):
    chem    = c.chemical.get_span().lower()
    disease = c.disease.get_span().lower()
    if (chem, disease) in CTD_lower and re.search(r'{{B}}.{0,20}caused by.{0,20}{{A}}', get_tagged_text(c), flags=re.I):
        return 1
    return 0

def LF_in_CDT_caused_by_2(c):
    chem    = c.chemical.get_span().lower()
    disease = c.disease.get_span().lower()
    if (chem, disease) in CTD_lower and re.search(r'{{B}}.*(caused|induced) by.*{{A}}', get_tagged_text(c), flags=re.I):
        return 1
    return 0

def LF_dash_induced(c):
    return 1 if re.search(r'{{A}}-induced {{B}}', get_tagged_text(c), flags=re.I) else 0

def LF_caused(c):
    return 1 if re.search(r'{{A}}.{0,20}caused.{0,20}{{B}}', get_tagged_text(c), flags=re.I) else 0

# Disease Tagging LFs Start Here

#### Document-Level Labeling Functions
We start with some labeling functions that label candidates based on document-level features.

#### Sentence-Level Labeling Functions
We also include some labeling functions that label candidates based on sentence-level features.

#### Mention-Level Labeling Functions
We now define a number of labeling functions that label candidates based on attributes related to the mention.

#### Dictionary Labeling Functions
We can use existing dictionaries for distant supervision.

#### Negative Labeling Functions
When writing labeling functions, it is important to provide negative supervision in addition to positive supervision.

We maintain a list of all LFs for convenience.

In [ ]:
# document and sentence-level
LFs_doc = [#LF_candidate_df,
           LF_known_abbreviation,
           LF_undefined_abbreviation
#            LF_in_title
]

LFs_sent = [#LF_in_list,
            LF_contiguous_mentions]

# mention-level LFs
LFs_true = [LF_tumors_growths,
#             LF_dep_path,
#             LF_acronym_def,
#             LF_long_phrase,
            LF_cancer,
            LF_disease_syndrome,
#             LF_deficiency,
            LF_indicators,
            LF_common_disease,
            LF_common_disease_acronyms,
            LF_deficiency_of,
            LF_come_with,
#             LF_contain_disease,
#             LF_parentheses,
#             LF_noun_phrases,
#             LF_type_x,
            LF_positive_indicator,
            LF_left_positive_argument,
#             LF_right_positive_argument,
            LF_right_negative_argument,
            LF_medical_afixes,
#             LF_dropped_syndrome
            LF_adj_diseases
           ]

LFs_dicts =  [#LF_DOID_human_disease_ontology,
#              LF_cell_molecular_dysfunction,
             LF_SNOWMED_CT_sign_or_symptom,
             LF_SNOWMED_CT_disease_or_syndrome,
             LF_MESH_disease_or_syndrome,
             LF_MESH_sign_or_symptom
            ]

LFs_false = [#LF_dangling_phrase,
             LF_chemical_name,
             LF_organs,
             LF_bodysym,
             LF_protein_chemical_abbrv,
             LF_has_punctuation,
             LF_gene_abbrv,
             LF_base_pair_seq, 
             LF_too_vague,
#              LF_weakness,
             LF_negation,
#              LF_activity,
#              LF_neg_prefix,
             LF_neg_surfix,
             LF_non_common_disease,
             LF_non_disease_acronyms,
#              LF_intensity_modifier,
             LF_pos_in,
             LF_gene_chromosome_link,
             LF_right_window_incomplete,
#              LF_left_negative_argument,
#              LF_not_disease
             LF_negative_indicator
            ]

LFs = LFs_true + LFs_false + LFs_dicts + LFs_doc + LFs_sent

## Applying Labeling Functions

First we construct a `CandidateLabeler`.

In [ ]:
from snorkel.annotations import LabelManager

label_manager = LabelManager()

Next we run the `CandidateLabeler` to to apply the labeling functions to the training `CandidateSet`.

In [ ]:
%time L_train = label_manager.create(session, train, 'LF Labels', f=LFs)
L_train

**OR** load if we've already created:

In [ ]:
%time L_train = label_manager.load(session, train, 'LF Labels')
L_train

Now say we want to add a new labeling function to our matrix:

In [ ]:
def LF_test_3(candidate):
    return -1 if random.random() < 0.2 else 0

We can also add or rerun a single labeling function (or more!) with the below command. Note that we set the argument `expand_key_set` to `True` to indicate that the set of matrix columns should be allowed to expand. 

Do this to test changes to the labeling functions.

In [ ]:
L_train = label_manager.update(session, train, 'LF Labels', True, f=[LF_test_3])
L_train

We can view statistics about the resulting label matrix:

In [ ]:
L_train.lf_stats()

## Fitting the Generative Model
We estimate the accuracies of the labeling functions without supervision. Specifically, we estimate the parameters of a `NaiveBayes` generative model.

In [ ]:
from snorkel.learning import NaiveBayes

gen_model = NaiveBayes()
gen_model.train(L_train)

In [ ]:
gen_model.w

In [ ]:
gen_model.save(session, 'Generative Params')

In [ ]:
gen_model.load(session, 'Generative Params')
gen_model.w

We now apply the generative model to the training candidates.

In [ ]:
train_marginals = gen_model.marginals(L_train)

## Training the Discriminative Model
We use the estimated probabilites to train a discriminative model that classifies each `Candidate` as a true or false mention.

In [ ]:
from snorkel.learning import LogReg

disc_model = LogReg()
disc_model.train(F_train, train_marginals, n_iter=1500, rate=1e-5)

In [ ]:
disc_model.w.shape

In [ ]:
%time disc_model.save(session, "Discriminative Params")

In [ ]:
w_prev = disc_model.w
%time disc_model.load(session, "Discriminative Params")
np.all(disc_model.w == w_prev)

## Evaluating on the Development `CandidateSet`

First, we create features for the development set.

Note that we use the training features feature set, because those are the only features for which we have learned parameters. Features that were not encountered during training, e.g., a token that does not appear in the training set, are ignored, because we do not have any information about them.

To do so with the `FeatureManager`, we call update with the new `CandidateSet`, the name of the training `AnnotationKeySet`, and the value `False` for the parameter `extend_key_set` to indicate that the `AnnotationKeySet` should not be expanded with new `Feature` keys encountered during processing.

In [ ]:
%time F_dev = feature_manager.update(session, dev, 'Train Features', False)

**OR** if we've already created one, we can simply load as follows:

In [ ]:
%time F_dev = feature_manager.load(session, dev, 'Train Features')

Next, we load the development set labels and gold candidates we made in Part III.

In [ ]:
L_dev = label_manager.load(session, dev, "CDR Development Labels -- Gold")

In [ ]:
gold_dev_set = session.query(CandidateSet).filter(CandidateSet.name == 'CDR Development Candidates -- Gold').one()

Now we can evaluate the discriminative model on the development set.

In [ ]:
tp, fp, tn, fn = disc_model.score(F_dev, L_dev, gold_dev_set, b=0.4)

## Viewing Examples
After evaluating on the development `CandidateSet`, the labeling functions can be modified. Try changing the labeling functions to improve performance. You can view the true positives, false positives, true negatives, and false negatives using the `Viewer`.

In [ ]:
from snorkel.viewer import SentenceNgramViewer

# NOTE: This if-then statement is only to avoid opening the viewer during automated testing of this notebook
# You should ignore this!
if not AUTOMATED_TESTING:
    sv = SentenceNgramViewer(tp, session, annotator_name="Tutorial Part IV User")
else:
    sv = None

In [ ]:
sv

In [ ]:
sv.get_selected()[0].parent

In [ ]:
sv.g

Next, in Part V, we will test our model on the test `CandidateSet`.